In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from config import feature_groups, feature_descriptions
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Read the data
# Note: You'll need to adjust the path based on your data structure
DATA_PATH = "../data/raw/"

# Load your dataset here
# Example:
train_X = pd.read_csv(DATA_PATH + "train_features.csv")
train_y = pd.read_csv(DATA_PATH + "train_labels.csv")
# train_y.head(3)

In [7]:
# Create DataFrame to show missing values statistics
missing_stats = []
for group, features in feature_groups.items():
    for feature in features:
        if feature in train_X.columns:
            missing = train_X[feature].isnull().sum()
            total = len(train_X)
            pct = (missing / total) * 100
            missing_stats.append({
                'Category': group,
                'Feature': feature,
                'Description': feature_descriptions.get(feature, ''),  # Add description column
                'Missing %': f"{pct:.2f}",
                'Missing Count': missing,
                'Total Count': total
            })

missing_df = pd.DataFrame(missing_stats)
missing_df['Missing %'] = missing_df['Missing %'].astype(float)
display(missing_df.sort_values('Missing %', ascending=False))
# missing_df.to_csv(DATA_PATH + "missing_values_stats.csv", index=False)

# Check the mean percentage of missing values for each group
# missing_df.groupby('Category')['Missing %'].mean().sort_values(ascending=False).to_csv(DATA_PATH + "missing_values_per_group.csv")

,Category,Feature,Description,Missing %,Missing Count,Total Count
165,Migration and U.S. Experience,a16a_12,"Year when respondent first left for the U.S., ...",99.27,3252,3276
167,Migration and U.S. Experience,a22_12,Main job type during longest stay in the U.S.,98.90,3240,3276
168,Migration and U.S. Experience,a33b_12,U.S. residency status,98.72,3234,3276
166,Migration and U.S. Experience,a21_12,Total years lived or worked in the U.S.,98.72,3234,3276
175,Employment and Job History,rjlocc_m_03,Category of their longest occupation,90.51,2965,3276
...,...,...,...,...,...,...
157,Health Insurance and Coverage,pem_def_mar_12,"Has health coverage with PEMEX, Defensa, or Ma...",2.72,89,3276
162,Health Insurance and Coverage,seg_pop_12,Has health coverage with Seguro Popular,2.72,89,3276
170,Housing and Environment,j11_12,Floor material of residence,2.29,75,3276
8,Demographic Information,ragender,Gender,0.00,0,3276


In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Get features with >= 70% missing values
high_missing_features = missing_df[missing_df['Missing %'] >= 70.0].sort_values('Missing %', ascending=False)

# Create a formatted string for pretty printing
print("\n=== Features with 70% or More Missing Values ===")
print(f"\nTotal Features: {len(high_missing_features)}")
print("\n{:<40} {:<10} {:<20}".format("Feature Name", "Missing %", "Category"))
print("-" * 70)

for _, row in high_missing_features.iterrows():
    print("{:<40} {:<10.1f} {:<20}".format(
        row['Feature'],
        float(row['Missing %']),
        row['Category']
    ))
print("\n" + "=" * 70)


=== Features with 70% or More Missing Values ===

Total Features: 9

Feature Name                             Missing %  Category            
----------------------------------------------------------------------
a16a_12                                  99.3       Migration and U.S. Experience
a22_12                                   98.9       Migration and U.S. Experience
a21_12                                   98.7       Migration and U.S. Experience
a33b_12                                  98.7       Migration and U.S. Experience
rjlocc_m_03                              90.5       Employment and Job History
rjob_end_03                              89.7       Employment and Job History
rjobend_reason_03                        89.4       Employment and Job History
rjob_end_12                              85.6       Employment and Job History
rjobend_reason_12                        85.1       Employment and Job History



In [5]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

selected_features = high_missing_features['Feature'].tolist()
merged_df = pd.merge(
    train_X[['uid'] + selected_features],
    train_y[['uid', 'year', 'composite_score']], 
    on=['uid'],
    how='inner'
)

# Distribution of the composite score (target)
# ==============================================
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=merged_df['composite_score'],
        nbinsx=30,
        name='Composite Score Distribution',
        opacity=0.7
    )
)

fig.update_layout(
    title='Distribution of Composite Scores',
    xaxis_title='Composite Score',
    yaxis_title='Count',
    width=800,
    height=500,
    showlegend=True
)

fig.show()
# ==============================================

# Print summary statistics for target variable
print("\nComposite Score (Target variable) Statistics:")
print("=" * 50)
stats = merged_df['composite_score'].describe()
print(f"{'Statistic':<15} {'Value':>10}")
print("-" * 50)
for stat_name, value in stats.items():
    print(f"{stat_name:<15} {value:>10.2f}")
print("=" * 50)


# Plot histograms for each feature
# ==============================================

# Create histograms
fig = make_subplots(
    rows=3, 
    cols=3,
    subplot_titles=[
        f"{feat}\n{feature_descriptions[feat]}\n)"
        for feat in selected_features
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# Color scheme
colors = {'Missing': 'rgba(255, 0, 0, 0.5)', 'Non-Missing': 'rgba(0, 0, 255, 0.5)'}

for idx, feature in enumerate(selected_features):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    missing_mask = merged_df[feature].isna()
    
    # Add histogram for non-missing values
    fig.add_trace(
        go.Histogram(
            x=merged_df[~missing_mask]['composite_score'],
            name='Non-Missing',
            opacity=0.7,
            nbinsx=30,
            marker_color=colors['Non-Missing'],
            showlegend=True if idx == 0 else False
        ),
        row=row, col=col
    )
    
    # Add histogram for missing values
    fig.add_trace(
        go.Histogram(
            x=merged_df[missing_mask]['composite_score'],
            name='Missing',
            opacity=0.7,
            nbinsx=30,
            marker_color=colors['Missing'],
            showlegend=True if idx == 0 else False
        ),
        row=row, col=col
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Composite Score", row=row, col=col)
    fig.update_yaxes(title_text="Count", row=row, col=col)

# Update layout
fig.update_layout(
    height=900,
    width=1200,
    title_text="Distribution of Composite Scores for Missing vs Non-Missing Values",
    barmode='overlay',
    showlegend=True
)

fig.show()
# ==============================================

# Print summary statistics for each feature
print("-" * 80)
for feature in selected_features:
    missing_mask = merged_df[feature].isna()
    missing_stats = merged_df[missing_mask]['composite_score'].describe()
    non_missing_stats = merged_df[~missing_mask]['composite_score'].describe()
    
    print(f"\n{feature} ({feature_descriptions[feature]}):")
    print(f"Missing values: {missing_mask.sum()} ({missing_mask.mean()*100:.1f}%)")
    print(f"Mean score (Missing): {missing_stats['mean']:.2f}")
    print(f"Mean score (Non-Missing): {non_missing_stats['mean']:.2f}")
    print(f"Score difference: {abs(missing_stats['mean'] - non_missing_stats['mean']):.2f}")
    print("-" * 80)


Composite Score (Target variable) Statistics:
Statistic            Value
--------------------------------------------------
count              4343.00
mean                157.02
std                  60.91
min                   4.00
25%                 114.00
50%                 157.00
75%                 200.00
max                 334.00


--------------------------------------------------------------------------------

a16a_12 (Year when respondent first left for the U.S., if they ever lived in the U.S.):
Missing values: 4309 (99.2%)
Mean score (Missing): 157.22
Mean score (Non-Missing): 131.32
Score difference: 25.90
--------------------------------------------------------------------------------

a22_12 (Main job type during longest stay in the U.S.):
Missing values: 4295 (98.9%)
Mean score (Missing): 157.31
Mean score (Non-Missing): 130.90
Score difference: 26.41
--------------------------------------------------------------------------------

a21_12 (Total years lived or worked in the U.S.):
Missing values: 4288 (98.7%)
Mean score (Missing): 157.32
Mean score (Non-Missing): 133.29
Score difference: 24.03
--------------------------------------------------------------------------------

a33b_12 (U.S. residency status):
Missing values: 4288 (98.7%)
Mean score (Missing): 157.32
Mean score (Non-Missing): 133.29
Score dif

# Based on the plot above, it seems that there is no real predictive power in the features with high missing values